# BUA 451 Final Project: Chicago Taxi Trip Data Analysis

**Author:** Connor McHugh         
**Date:** April 29th, 2025  
**Course/Project:** BUA 451: Advanced Business Analytics  
**Dataset:** Chicago Taxi Trips (Big Query)


# Executive Summary

**Objective:**
* The objective of this project was to pull out meaningful insights for Chicago-based taxi companies in order to help them make better decisions. These decisions include where they should place their fleets, what time of day yields the most tips for their drivers, and which hours of the day have the most passenger volumes.

**Data:**
* The data for this project utilizes the Chicago Taxi Trips dataset from BigQuery, focusing on trip details, fares, tips, payment types, and pickup/dropoff locations, amongst other things. The data for this project starts in April of 2017 and goes to the present day, being updated every Sunday. Some of the key variables used include total trip fare, pickup neighborhood, the pickup and drop off time stamps, and tip amount.

**Methodology:**
* The methodology I used for analysis was putting myself in the shoes of taxi company owners for whom this data would be helpful. I thought about what they would care about if they were paying someone to analyze this data for them. From this, I thought that doing exploratory data analysis on volume numbers and trip fare amounts would be valuable, as that is what would be important to the owner of a taxi company. To clean the data, NOT NULL was put into queries where applicable. Furthermore, many numbers had to be changed to numeric or other data types to allow for graphing. As for the graphs, I based them on the time of the trip, as the time of the day can be incredibly important for a taxi driver.

**Key Findings:**
*  I found that the average fare and tip are the highest during the morning, while the afternoon has much higher volumes of passengers. This could mean fewer taxis are available in the morning, thus allowing them to charge a higher fare, and more available in the afternoon for the higher volume. I also found that payment via cash was the most popular payment type for customers, but payment via credit card led to the highest amount of tips. Next, I found that Downtown Chicago ( Near North Side and the Loop) has the highest total passenger pickups by fair, with the other popular pickup locations still being in the 10s millions. Also, from my logistic regression model, I found that the company can heavily influence the payment type used.

**Recommendations:**
* From these findings, I recommend that a taxi company have the most drivers ready early in the morning and later in the afternoon. Fares are the highest in the morning, so it would make sense to capitalize on those higher fares for the most profit. Furthermore, the afternoon has the highest total volume of passengers. While the fares may not be as high, the volume is much higher, still allowing for sufficient profit to be made. Also, I recommend that a company accept credit card payments as much as possible. Fares and tips via this payment type tend to be higher than any other payment type, allowing the driver and company to maximize profit. Lastly, I recommend a company that looks into serving the neighborhoods outside the downtown area. These areas still have 10s of millions of riders that may feel underserved, so a company could make a nice living serving these customers.

In [ ]:
!pip install pandas-gbq --quiet
!pip install google-cloud-bigquery pandas
!pip install plotly.express --quiet
!pip install ipywidgets --quiet
!pip install imbalanced-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

#Library Info

**Core Libraries**

* Pandas -> Used for data manipulation and analysis

* Matplotlib.pyplot -> Plotting library for interactive plots

* Seaborn -> Another plotting library

* Google.cloud.bigquery -> Enables interaction with Big Query

* Plotly.express -> High level interface for interactive plotting

* Plotly.graph_objects -> Same as express, just a little more detailed

* Numpy -> Support for numerical operations with arrays and matrices

**Machine Learning Libraries**

* sklearn.model_selection.train_test_split -> Used to split data into training and testing sets for evaluating machine learning models.

* sklearn.linear_model.LogisticRegression -> Provides an implementation of the logistic regression algorithm for binary classification tasks

* sklearn.metrics.accuracy_score -> Used to calculate the accuracy of a classification model's predictions

* sklearn.preprocessing.OneHotEncoder, StandardScaler -> OneHotEncoder converts categorical features into binary variables, while StandardScaler performs a a Z-Score transformation to make it so all the numbers are small and close to 0. These two make testing and training our model better.

* sklearn.impute.SimpleImputer -> Used to impute missing values in a dataset with the mean, median, etc

* imblearn.over_sampling.SMOTE -> Used to balance data for model training

* imblearn.over_sampling.RandomOverSampler -> used to balance the data and over sample it
* sklearn.metrics.classification_report -> Library for evaluating supervised learning models

In [ ]:
import pandas as pd
from pandas.io import gbq
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report

##Connecting to my Google Cloud Project so I can access Big Query, testing my connection to Big Query, and testing my connection to my desired dataset

In [ ]:
project_id = "bua-451-final-project-457020"
client = bigquery.Client(project = project_id)

In [ ]:
test_query = "SELECT CURRENT_DATE() as today"

# Run the query using the BigQuery client
test_result = client.query(test_query).result().to_dataframe()

print("Connection successful! Today's date from BigQuery is:", test_result['today'][0])

Connection successful! Today's date from BigQuery is: 2025-12-28


##Testing the connection to my desired table

In [ ]:
query = """
SELECT taxi_id
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
LIMIT 1
"""

try:
    result = client.query(query).result().to_dataframe()
    print("Connected! Sample taxi_id from table:", result['taxi_id'][0])
except Exception as e:
    print("Connection failed or table not accessible.")
    print("Error:", e)

Connected! Sample taxi_id from table: f760c301740afe577d6c0e2f6de2cfecccf6ede158b8a3825331005da37a008716599169f4b467bc8b88e65318b5008499ad0ea4e05a7465c6bce4a1d5e70350


##Dataset Description

**Dataset Name:** Chicago Taxi Trips

**Source:** Big Query

**Description:** This dataset is a comprehensive view of Chicago's taxi activity between April 2017 and the present day. This dataset is still being updated, with updates occurring every Sunday. The table with all the data is named "taxi trips." This is the table that gets updated. Chicago Taxi Trips is the name of the dataset, but it does not get updated weekly. This is because this dataset only includes the "taxi trip" table, which gets its information updated weekly. The table includes information like total trip distance, pickup, and dropoff location, and total trip fare, amongst other things.

**Key Variables:**
* `trip_start_timestamp`: The timestamp indicating when the trip started.
* `trip_end_timestamp`: The timestamp indicating when the trip ended.
* `trip_seconds`: The duration of the trip in seconds.
* `trip_miles`: The total distance of the trip in miles.
* `pickup_community_area`: The community area where the trip originated.
* `dropoff_community_area`:  The community area where the trip ended.
* `fare`: The base fare for the trip.
* `tips`: The amount of tip given to the driver.
* `payment_type`: The method used for payment (e.g., cash, credit card).


**Data Period:** April 12, 2017 - April 27, 2025

**Data Update Frequency:** Weekly, every Sunday

**Data Limitations:**
* The dataset may contain missing values for some variables.
* The accuracy of the data relies on the information provided by taxi drivers and passengers.
* Many of the numbers are in different datatypes, requiring much preprocessing before they are usable. This makes troubleshooting more difficult as well.
* Taxi trips may be influenced by external events, such as weather or concerns, that cannot be easily accounted for during the analysis.
* There may be pickup neighborhoods that may not get served due to low ease of access.

###**Exploratory Data Analysis**

Viewing the table structure

In [ ]:
query = """
SELECT *
FROM `bigquery-public-data`.chicago_taxi_trips.taxi_trips
WHERE trip_start_timestamp < '2025-04-30'
LIMIT 10
"""

df = client.query(query).result().to_dataframe()
print(df)

                                 unique_key  \
0  18701dbeb233657ebf5b805185d3619a9184ae82   
1  2e5a5c7566151a56e6a43cf4a31b8dfede6dfdac   
2  23dce7780f8884435f81a6319fb406f4ed7aaca6   
3  2adf23dec9ab5c70a69f8d52cb4506d7204246ca   
4  2e7d76f5b4a4ecc42c598dc402318e3163d7b02a   
5  29c9599fd7209fe348263f096934e41c4d36df8f   
6  9b236aa3ac94a999073f57b4a50f2dc24ee497a2   
7  25689c1cd2237a40e2a3a83a7785d3fa949c89c5   
8  77e3211cacb518d70a314dd80fea28e5934b1f75   
9  26bfc0734c257edaf860c3d9d42eb1097b290477   

                                             taxi_id  \
0  4227951e0b730645c34ef273718eb41afb843f7bf33bc4...   
1  b817ee2ac77f0411dbf2976cd3883053dc69380b8d61c7...   
2  a109fba2e815112c26832ff506072be742be30a295285a...   
3  aa3a04f31f86b04a0f9456d9e93f6699715392ba9e5c1a...   
4  ae49305993017ae37676aa741c7c8147612a9f6e4384ba...   
5  12bfcfb8715a63cacb9a4bc127f622071bab27b36a8488...   
6  0c8698dd0e6d968d193771b14459e0ad120684b5fb14d0...   
7  77b0baaf4e271f6fadb2eaf15ab5feb

# **All Data for this project will be from April 12, 2017 to April 27, 2025. This data is updated weekly, every Sunday night. The dates will not be modified at all, as it will allow for the best insights and training of the model later on**       


# **EDA Results and Visualization 1: Average Tip and Fare by hour of day**

The first graph that will be produced shows the average tip, average trip fare, and average duration of the trip. The goal is to see which hour of the day has the highest tip and total trip fare amount. This will help a taxi company determine which hours of the day are most profitable for both the company and the drivers.

In [ ]:
query = """
SELECT
    EXTRACT(HOUR FROM trip_start_timestamp) AS trip_hour,
    ROUND(AVG(tips),2) AS avg_tip_amount,
    ROUND(AVG(trip_total),2) AS avg_fare,
    ROUND(AVG(trip_seconds)/60,2) AS avg_trip_duration
FROM
    `bigquery-public-data`.chicago_taxi_trips.taxi_trips
WHERE tips IS NOT NULL AND trip_total IS NOT NULL AND trip_seconds IS NOT NULL
AND trip_start_timestamp < '2025-04-30'
GROUP BY
    trip_hour
ORDER BY
    trip_hour;
"""

df = client.query(query).result().to_dataframe()
print(df.head(100))

    trip_hour  avg_tip_amount  avg_fare  avg_trip_duration
0           0            1.28     15.26              10.89
1           1            1.18     14.27              10.23
2           2            1.06     13.35               9.62
3           3            1.04     13.43               9.80
4           4            1.26     16.43              11.07
5           5            1.79     21.92              14.08
6           6            1.79     21.31              15.53
7           7            1.49     16.89              14.59
8           8            1.41     15.29              13.98
9           9            1.45     15.77              13.64
10         10            1.46     16.40              13.10
11         11            1.49     16.65              13.10
12         12            1.52     16.76              13.54
13         13            1.55     16.98              14.04
14         14            1.58     17.36              14.95
15         15            1.60     17.64              16.

The code for the plot below shows the average tip amount and fare based on the hour of the day. One thing to note here is that there are 2 12ams. Due to how the code was written and how the logic determines am or pm, I could not get 12 pm and 12 am in the correct spots. This was troubleshooted for about 45 minutes, but no resolution was found. If you hover your cursor over any of the lines, it will tell you what that line is and the value of that line. This is particularly helpful for viewing tips, as they do not change nearly as much as the trip fare.

In [ ]:
#Plotting tip analysis based on the hour of the day
fig = px.line(df, x='trip_hour', y=['avg_tip_amount', 'avg_fare', 'avg_trip_duration'],
              title='Tip Analysis and Average Fare by Hour of Day')

fig.update_xaxes(tickvals=df['trip_hour'],
                 ticktext=[f'{hour:.0f}am' if hour <= 12 else f'{hour-12:.0f}pm' for hour in df['trip_hour']])

x_axis_labels = [f'{hour:.0f}am' if hour <= 12 else f'{hour-12:.0f}pm' for hour in df['trip_hour']]
x_axis_labels = ['12am' if label == '0am' else '12pm' if label == '0pm' else label for label in x_axis_labels]  # Update labels

fig.update_xaxes(tickvals=df['trip_hour'], ticktext=x_axis_labels)

fig.update_layout(
    xaxis_title='Hour of Day',
    yaxis_title='Amount',
    legend_title='Metrics',
    legend=dict(
        title='Metrics',
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bordercolor='Black',
        borderwidth=2
    )
)

for i, trace in enumerate(fig.data):
    if trace.name == 'avg_tip_amount':
        trace.name = 'avg_tip_amount ($)'
    elif trace.name == 'avg_fare':
        trace.name = 'avg_fare ($)'
    elif trace.name == 'avg_trip_duration':
        trace.name = 'avg_trip_duration (min)'

fig.show()

**Analysis**

* The average trip fare is at its highest between 5 a.m. and 6 a.m. and 3 p.m.
 Early morning has the highest average fare by far, as there is a rush to get to work on time. People funnel out of the office at the end of the day, so the fare is not as significant. However, almost everyone needs to be in the office by 7/8 a.m., leading to a higher demand in the morning for a scarce resource, thus the reason for the higher fare. Furthermore, drivers are tipped more during this time, around 1.80 per trip, compared to the average of around 1.50 throughout the rest of the day. The fares stay stable throughout the day, dropping into the later hours of the night as there is less demand from drivers and a shorter trip duration. The shorter trip duration is likely due to people coming home from a night out and choosing to go somewhere near their homes to minimize the travel distance.

* In conclusion, if a taxi company wants to position itself to make the most money per trip, it should be ready to have drivers between 5 a.m. and 7 a.m. The rest of the day hovers around the same fares, peaking at 3 p.m., so a company should capitalize on these higher fares when possible. While the volume in the morning is not as high as the volume in the afternoon, on a per-trip basis, the morning is more profitable.


#**EDA Results and Visualization 2: Total passengers by hour of day**

In [ ]:
query = """
    SELECT
        EXTRACT(HOUR FROM trip_start_timestamp) AS trip_hour,
        COUNT(*) AS trip_frequency,
        SUM(trip_total) AS total_fare_amount
      FROM
        `bigquery-public-data`.chicago_taxi_trips.taxi_trips
      WHERE trip_total IS NOT NULL
      AND trip_start_timestamp < '2025-04-30'
      GROUP BY
        trip_hour
      ORDER BY
        trip_hour;
    """

trip_frequency_df = client.query(query).result().to_dataframe()
print(trip_frequency_df)

    trip_hour  trip_frequency  total_fare_amount
0           0         7011757       1.071397e+08
1           1         5792123       8.282239e+07
2           2         4571284       6.110260e+07
3           3         3410744       4.587894e+07
4           4         2471341       4.058554e+07
5           5         2233139       4.893853e+07
6           6         3088086       6.598331e+07
7           7         5641407       9.551410e+07
8           8         8698179       1.331888e+08
9           9        10001055       1.578126e+08
10         10         9750701       1.600955e+08
11         11        10276288       1.713134e+08
12         12        11053095       1.854680e+08
13         13        11269634       1.915770e+08
14         14        11246403       1.954731e+08
15         15        11450737       2.022033e+08
16         16        12121608       2.085579e+08
17         17        13123465       2.147927e+08
18         18        13889057       2.191939e+08
19         19       

The code below will create a bar chart that shows the total number of passengers based on the hour. The same error was encountered where I could not turn my 12 a.m. into 12.pm. This bar chart is also accompanied by a table showing the total fare amount based on the hour. This was done to pull out insights based on the total fare amount and time. For example, there is a mmore passengers at 3 a.m. and 6 a.m., yet 6 a.m. has almost 10 million more in total fares.

In [ ]:
fig = px.bar(trip_frequency_df, x='trip_hour', y='trip_frequency',
             title='Trip Frequency by Hour of Day')

x_axis_labels = ['12am' if label == '0am' else '12pm' if label == '0pm' else label for label in x_axis_labels]  # Update labels

fig.update_xaxes(
    tickvals=trip_frequency_df['trip_hour'],
    ticktext=[f'{hour:.0f}am' if hour <= 12 else f'{hour-12:.0f}pm' for hour in trip_frequency_df['trip_hour']]
)

fig.update_xaxes(tickvals=trip_frequency_df['trip_hour'], ticktext=x_axis_labels)

fig.update_layout(yaxis_title='Total Passengers')


fig.update_layout(title='Total Passengers by Hour of Day')

fig.show()


trip_frequency_df['total_fare_amount'] = pd.to_numeric(trip_frequency_df['total_fare_amount'])

trip_frequency_df['Hour of Day'] = trip_frequency_df['trip_hour'].apply(lambda hour: f'{hour:.0f}am' if hour <= 12 else f'{hour-12:.0f}pm')

trip_frequency_df['Hour of Day'] = trip_frequency_df['Hour of Day'].replace({'0am': '12am', '0pm': '12pm'})

formatted_table = trip_frequency_df.set_index('Hour of Day')[['total_fare_amount']].style.format({
    'total_fare_amount': '{:,.2f}'
})

display(formatted_table)



,total_fare_amount
Hour of Day,
12am,"107,139,672.08"
1am,"82,822,393.31"
2am,"61,102,599.47"
3am,"45,878,937.24"
4am,"40,585,542.79"
5am,"48,938,529.47"
6am,"65,983,313.54"
7am,"95,514,103.86"
8am,"133,188,753.26"


**Analysis**
* 5 p.m.-7 p.m. has the highest total passenger count over any 3 hours in the day. This range also includes the three highest total fare amounts based on the hour of the day. A company must have the most drivers ready to drive at this time, as it has the highest chance of having numerous passengers in a row and not having to drive around with an empty taxi. Drivers lose money any time they drive without a passenger, as the only way they get paid is if they have a paying customer in their cab. Maximizing the time a driver has a passenger is in the driver's best interest. Furthermore, a company can be prepared not to have as many drivers ready in the late hours/ early hours of the night/morning. The lower fare amount, mixed with the lower frequency, does not make it worth having a large fleet of drivers ready.

* One interesting thing that this chart shows is that the most profitable time of the day is between the hours of 5 p.m. and 7 p.m., from a volume perspective, even though the average fare is not as high as earlier in the day. This is because the total number of passengers using taxis during this time is significantly higher than in the morning, leading to a lot more money moving around the city at these times. This shows that the most profitable time to be a driver is earlier in the morning since you are getting tipped more per customer and later in the afternoon since there is a larger volume of passengers. If a taxi company can only have 1 time of the day that it can run its fleet, it should be in the evening, between 5 p.m. and 8 p.m.  

# **EDA Results and Visualization 3: Most popular pickup locations**
The below code is run to turn area mapping numbers into place names. The data from the Chicago taxi dataset organized geographic locations by community codes. For example, Downtown Chicago may have a community code of 52. While this may be good for query and record keeping, it's not the best for visualization and pulling insights. This line is run to match the names of the communities with the community code, which is matched with the query data later on.

In [ ]:
area_mapping = {
    1: 'Rogers Park',
    2: 'West Ridge',
    3: 'Uptown',
    4: 'Lincoln Square',
    5: 'North Center',
    6: 'Lake View',
    7: 'Lincoln Park',
    8: 'Near North Side',
    9: 'Edison Park',
    10: 'Norwood Park',
    11: 'Jefferson Park',
    12: 'Forest Glen',
    13: 'North Park',
    14: 'Albany Park',
    15: 'Portage Park',
    16: 'Irving Park',
    17: 'Dunning',
    18: 'Montclare',
    19: 'Belmont Cragin',
    20: 'Hermosa',
    21: 'Avondale',
    22: 'Logan Square',
    23: 'Humboldt Park',
    24: 'West Town',
    25: 'Austin',
    26: 'West Garfield Park',
    27: 'East Garfield Park',
    28: 'Near West Side',
    29: 'North Lawndale',
    30: 'South Lawndale',
    31: 'Lower West Side',
    32: 'The Loop',
    33: 'Near South Side',
    34: 'Armour Square',
    35: 'Douglas',
    36: 'Oakland',
    37: 'Fuller Park',
    38: 'Grand Boulevard',
    39: 'Kenwood',
    40: 'Washington Park',
    41: 'Hyde Park',
    42: 'Woodlawn',
    43: 'South Shore',
    44: 'Chatham',
    45: 'Avalon Park',
    46: 'South Chicago',
    47: 'Burnside',
    48: 'Calumet Heights',
    49: 'Roseland',
    50: 'Pullman',
    51: 'South Deering',
    52: 'East Side',
    53: 'West Pullman',
    54: 'Riverdale',
    55: 'Hegewisch',
    56: 'Garfield Ridge',
    57: 'Archer Heights',
    58: 'Brighton Park',
    59: 'McKinley Park',
    60: 'Bridgeport',
    61: 'New City',
    62: 'West Elsdon',
    63: 'Gage Park',
    64: 'Clearing',
    65: 'West Lawn',
    66: 'Chicago Lawn',
    67: 'West Englewood',
    68: 'Englewood',
    69: 'Greater Grand Crossing',
    70: 'Ashburn',
    71: 'Auburn Gresham',
    72: 'Beverly',
    73: 'Washington Heights',
    74: 'Mount Greenwood',
    75: 'Morgan Park',
    76: "O'Hare",
    77: 'Edgewater'
}

Here, the query is limited to only 10 results. The 50th highest pickup location most likely will not provide me any insights, so I do not need to see it. Using that logic, I limit my output to the top 10 pickup locations

In [ ]:
query = """
SELECT
    pickup_community_area,
    COUNT(*) AS pickup_count
  FROM
    `bigquery-public-data`.chicago_taxi_trips.taxi_trips
  WHERE pickup_community_area IS NOT NULL
  AND trip_start_timestamp < '2025-04-30'
  GROUP BY
    pickup_community_area
  ORDER BY
    pickup_count DESC
  LIMIT 10;
"""

pickup_location_df = client.query(query).result().to_dataframe()
print(pickup_location_df)

pickup_location_df['pickup_community_area'] = pickup_location_df['pickup_community_area'].map(area_mapping)

fig = px.bar(
    pickup_location_df,
    x='pickup_community_area',
    y='pickup_count',
    title='Top 10 Pickup Locations',
    labels={'pickup_community_area': 'Pickup Community Area', 'pickup_count': 'Number of Pickups'}
)
fig.show()

   pickup_community_area  pickup_count
0                      8      59856637
1                     32      43451725
2                     28      17760419
3                     76      14755694
4                      6      10865172
5                      7       8372811
6                     24       5455210
7                     33       4855880
8                     56       3710334
9                      3       2898171


**Analysis**
* Most of the pickups within the city occur in the Near North Side community and the Loop community. These two communities are located in the downtown area of Chicago, so it makes the most sense that these areas have the highest volume of pickups. Many people live in work in downtown cities areas, often without cars, leading to a higher proportion of the population relying on taxis. A taxi company should have a higher proportion of their fleet in these two areas to capture the most amount of business they can. However, taxi companies should not overlook the other areas of the city. Companies should not ignore large frequencies, in the tens of millions, in these more suburban communities.


# **EDA Results and Interactive Visualization: Customer Payment Preferences**

This next query determines the payment preferences of passengers. Essentially, it groups each payment type and determines how many passengers paid with that payment type and how much money went through that payment type

In [ ]:
query = """
SELECT
    payment_type,
    COUNT(*) AS user_count,
    CAST(SUM(trip_total) AS BIGNUMERIC) AS total_money_spent
  FROM
    `bigquery-public-data`.chicago_taxi_trips.taxi_trips
  WHERE payment_type IS NOT NULL AND trip_total IS NOT NULL
  AND trip_start_timestamp < '2025-04-30'
  GROUP BY
    payment_type
  ORDER BY
    user_count DESC; # Use payment_count for ordering
"""

payment_preference_df = client.query(query).result().to_dataframe()
print(payment_preference_df)

   payment_type  user_count                                  total_money_spent
0          Cash   118835065  1487031046.91995906829833984375000000000000000000
1   Credit Card    85542429  1840268152.98978543281555175781250000000000000000
2        Mobile     2583320    51223397.77999991923570632934570312500000000000
3        Prcard     2189975    50727300.26000031083822250366210937500000000000
4       Unknown     1529757    30300143.90999998897314071655273437500000000000
5     No Charge      823110    13127819.26999998278915882110595703125000000000
6       Dispute       88475     1429315.53999999910593032836914062500000000000
7         Pcard       36874      373051.71000000037020072340965270996093750000
8         Split        3442       64668.43000000000756699591875076293945312500
9       Prepaid        1813       40650.02999999999155988916754722595214843750
10     Way2ride         142        2362.78999999999996362021192908287048339844


Next, the values are log-transformed and changed to numeric for better graphing. Without the log scale, the axis looks messy, and the graph isn't very usable. The log scale allows for a better comparison between the different payment types.

In [ ]:
payment_preference_df["total_money_spent"] = pd.to_numeric(payment_preference_df["total_money_spent"])
payment_preference_df["log_user_count"] = np.log1p(payment_preference_df["user_count"])
payment_preference_df["log_total_money_spent"] = np.log1p(payment_preference_df["total_money_spent"])


fig = go.Figure(
    data=[
        go.Bar(
            name="Total Money Spent (Log Scale)",
            x=payment_preference_df["payment_type"],
            y=payment_preference_df["log_total_money_spent"],
            offsetgroup=0,
            text=[f'{x:,.0f}' for x in payment_preference_df["total_money_spent"]],
            textposition="outside",
        ),
        go.Bar(
            name="User Count (Log Scale)",
            x=payment_preference_df["payment_type"],
            y=payment_preference_df["log_user_count"],
            offsetgroup=0,
            text=[f'{x:,.0f}' for x in payment_preference_df["user_count"]],
            textposition="outside",
        )
    ],
    layout={
        "yaxis": {
            "title": "Log Scale Value",
            "type": "log"
        },
        "xaxis": {"title": "Payment Type"},
        "title": "Payment Type Preferences (Log Scale)",
        "barmode": "overlay"
    },
)
fig.show()


**Analysis**
* This bar chart shows users' payment preferences. The red chart shows how many people paid via that payment type, and the blue is the total money spent by that payment method. You can click on the legend and select which bar to view. Once you do this, you will see the total amount (nonlog scaled) displayed above the line. The axis is displayed on a log scale to make the chart better for analysis. This is the reason for the gap between 10 and 6 on the axis. One of the charts, the difference in magnitude is so large, to the point where the axis gets wierd to reflect this magnitude difference. This is on the user count graph, as it goes from 86 million people using credit card to 4 million people using mobile payment.
* This chart tells us that even though more people pay with cash, more money is spent via credit card. About 30 million more users use cash over credit cards, but credit card users spend almost 500 million more on their fares. This is likely because credit card users are more likely to tip than any other payment type by a wide margin (see graph below). To maximize the amount of money a user spends on each trip, taxi companies should welcome using credit cards. Furthermore, a taxi company can only consider accepting credit cards or mobile payments to induce people to spend more money. Mobile payment types such as Cash App, Zelle, and Venmo are rapidly gaining popularity, so companies should be ready to adapt.


The below chart is a supporting bar chart for the chart above it. It helps explain why credit card users spend so much more than cash users, even though there are fewer credit card users. Credit card users tip a lot more than any other payment type, by far. This explains why we see the total amount spent on credit cards being so much higher than cash despite having fewer users.

In [ ]:
query = """
SELECT
    payment_type,
    SUM(tips) AS total_tips
  FROM
    `bigquery-public-data`.chicago_taxi_trips.taxi_trips
  WHERE tips IS NOT NULL
  AND trip_start_timestamp < '2025-04-30'
  GROUP BY
    payment_type
  ORDER BY
    total_tips DESC;
"""

tips_by_payment_type_df = client.query(query).result().to_dataframe()
fig = px.bar(
    tips_by_payment_type_df,
    x='payment_type',
    y='total_tips',
    title='Total Tips by Payment Type',
    labels={'payment_type': 'Payment Type', 'total_tips': 'Total Tips ($)'}
)

fig.show()

# **Predictive Modeling 1: Logistic Regression: Likelihood of getting a tip**

This next cell determines the likelihood of getting a tip based on several features, including the total trip cost, the length of the trip, the number of miles traveled, the hour of the trip, and the payment type used. I set my random seed as 42 for splitting the data since I felt like it. This ensures a reproducible split every single time the code is run. The model uses a logistic regression model to determine the probability of belonging to a particular class.

First I check to see if I need to use smote to rebalance my data. The proportions are around the same magnitude, so rebalancing is not needed. (Since its binary, 0 gets assigned to the alphabetically first class. We have no and yes, so no gets assigned to 0 and yes gets assigned to 1)

In [ ]:
query = """
SELECT
    tips,
    trip_total,
    trip_seconds,
    trip_miles,
    payment_type,
    trip_start_timestamp
FROM
    `bigquery-public-data`.chicago_taxi_trips.taxi_trips
WHERE
    tips IS NOT NULL AND
    trip_total IS NOT NULL AND
    trip_seconds IS NOT NULL AND
    trip_miles IS NOT NULL AND
    payment_type IS NOT NULL AND
    trip_start_timestamp < '2025-04-30'
LIMIT 10000
"""
taxi_df = client.query(query).result().to_dataframe()
taxi_df['tip_given'] = (taxi_df['tips'] > 0).astype(int)
print(taxi_df['tip_given'].value_counts(normalize=True))

tip_given
1    0.7298
0    0.2702
Name: proportion, dtype: float64


In [ ]:
#Limit query to 10,000 observations
query = """
SELECT
    tips,
    trip_total,
    trip_seconds,
    trip_miles,
    payment_type,
    trip_start_timestamp
FROM
    `bigquery-public-data`.chicago_taxi_trips.taxi_trips
WHERE
    tips IS NOT NULL AND
    trip_total IS NOT NULL AND
    trip_seconds IS NOT NULL AND
    trip_miles IS NOT NULL AND
    payment_type IS NOT NULL AND
    trip_start_timestamp < '2025-04-30'
LIMIT 10000
"""
taxi_df = client.query(query).result().to_dataframe()

taxi_df['trip_hour'] = pd.to_datetime(taxi_df['trip_start_timestamp']).dt.hour
taxi_df['tip_given'] = (taxi_df['tips'] > 0).astype(int)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_payment_type = encoder.fit_transform(taxi_df[['payment_type']])
encoded_df = pd.DataFrame(encoded_payment_type, columns=encoder.get_feature_names_out(['payment_type']))
taxi_df = taxi_df.drop('payment_type', axis=1)
taxi_df = pd.concat([taxi_df, encoded_df], axis=1)

target_variables = ['tips', 'tip_given']
taxi_df = taxi_df[[c for c in taxi_df if c not in target_variables] + target_variables]

X = taxi_df[taxi_df.columns[:-2]].drop('trip_start_timestamp', axis=1)
y = taxi_df[taxi_df.columns[-1]].values

#Scaling my features before spliutting my data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model training
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred) * 100

print("Model Coefficients (Weights):")

feature_names = encoder.get_feature_names_out(['payment_type'])
all_feature_names = ['trip_total', 'trip_seconds', 'trip_miles', 'trip_hour'] + list(feature_names)

for feature_name, weight in zip(all_feature_names, model.coef_[0]):
    print(f"{feature_name}: {weight:.4f}")

print(f"\nIntercept (Bias): {model.intercept_[0]:.4f}")
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Model Coefficients (Weights):
trip_total: -0.3099
trip_seconds: 0.0325
trip_miles: 0.1450
trip_hour: 0.1611
payment_type_Cash: -1.7260
payment_type_Credit Card: 1.5953
payment_type_Dispute: -0.0749
payment_type_Mobile: 0.1532
payment_type_No Charge: 0.2904
payment_type_Prcard: -0.1655
payment_type_Unknown: -0.2218

Intercept (Bias): 0.7496
Accuracy: 91.25
              precision    recall  f1-score   support

           0       0.99      0.67      0.80       521
           1       0.90      1.00      0.94      1479

    accuracy                           0.91      2000
   macro avg       0.94      0.83      0.87      2000
weighted avg       0.92      0.91      0.91      2000



**Analysis**
* Supervised learning was applied to determine the likelihood of getting a tip based on the total trip amount, the trip time, and the hour of the trip, amongst other things. The model was able to predict the outcome of a tip with 91.25% accuracy. This high accuracy is likely due to the fact that there is a larger proportion of no-tip givens, leading the model to have a vast amount of data to predict when a driver will not get tipped. The highest contributors towards getting a tip are getting paid by credit card or mobile payment and how long the trip was. The highest things against getting a tip are getting paid in cash or by how short the trip was.

* Precision, recall, and F-1 scores are all high, indicating the overall effectiveness of this model. This high performance is likely due to the large amount of training data, allowing the model to see numerous transactions resulting in tips/no tips. The precision on "no tip" is incredibly high, meaning a low number of false negatives for this class. Furthermore, the F1 score indicates that the model is equally as good at identifying both class categories.

* This shows that tip is more dependent on the payment type than anything else. When paying with a credit card, it's often easier to include a few extra cents of a tip, as there may be an option on the screen to tip. With cash, there is no easy screen that shows tip percentages, so the passenger has to figure out how much of a tip to give manually. Furthermore, many cash users may just let the driver keep the change as their tip, which would result in many smaller tips.

* Furthermore, when the payment method is Unknown, it's likely to result in no tips. This is because the "Unknown" payment types include rides that may have been canceled due to various reasons and rides that were free (training or promotional rides). These rides result in the passenger being upset with the taxi driver or not needing to tip them due to the nature of the ride. This leads to no tip given in these situations, thus the reason for the negative correlation.

## **Managerial Insights**

**Key Findings:**
* Early morning trips (5 a.m.-7 a.m.) command the highest average fares, while evening rush hour (5 p.m.-7 p.m.) sees the highest trip volume.
* Credit card payments contribute to higher overall revenue than cash, largely due to increased tipping.
* Downtown areas, particularly Near North Side and The Loop, are the most popular pickup locations.
* Specific taxi companies show stronger associations with certain payment types, suggesting potential marketing or operational strategies.

**Takeaways:**
* **Demand and fares fluctuate throughout the day, requiring strategic driver allocation. A business must determine what metric (volume or revenue per tip) to prioritize to make the best strategic decision**
* **Credit card adoption is crucial for maximizing revenue and tip collection.**
* **While Downtown areas present the highest opportunities for passenger acquisition, the suburban areas should not be ignored. A company can make it their whole business model to serve these suburban customers and make a good living off of it**
* **Company-specific payment preferences should be leveraged for targeted marketing. Certain taxi companies may only accept certain payment types. These companies can advertise their services to users of this payment type, encouraging customer growth.**

**Recommendations:**
* Optimize driver availability during peak demand and high-fare periods to maximize revenue potential.
* Incentivize credit card usage through promotions or loyalty programs to encourage higher spending and tips.
* Focus marketing efforts on high-volume pickup areas and consider strategic partnerships with businesses in these locations.
* Tailored payment options or promotions for specific companies could be explored to enhance customer experience and revenue.

**Implications for Decision-Making:**
* These insights can inform driver scheduling, pricing strategies, marketing campaigns, and payment system choices to optimize profitability and customer satisfaction.
* Understanding passenger behavior and payment preferences can enable more targeted services and promotions.

**Future Directions:**
* Investigating the impact of external factors like weather and events on trip patterns could further refine operational strategies.
* Analyzing passenger demographics and preferences could unlock personalized services and marketing opportunities.
* Exploring the use of predictive models to forecast demand and optimize resource allocation could enhance operational efficiency.